In [9]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# 1. Veri Çekme (BIST 100 eklendi)
try:
    print("Fetching EREGL.IS data...")
    eregl = yf.Ticker("EREGL.IS")
    eregl_df_raw = eregl.history(period="3y")
    if eregl_df_raw.empty or 'Close' not in eregl_df_raw.columns:
        print("❌ Hata: EREGL.IS için fiyat verisi ('Close') çekilemedi.")
        eregl_df = pd.DataFrame()
    else:
        # Timezone kaldırarak birleştir
        eregl_df = eregl_df_raw['Close'].tz_localize(None).to_frame(name='EREGL_Fiyat')
        print("✅ EREGL.IS verisi çekildi.")

    print("\nFetching TRY=X data...")
    usdtry = yf.Ticker("TRY=X")
    usdtry_df_raw = usdtry.history(period="3y")
    if usdtry_df_raw.empty or 'Close' not in usdtry_df_raw.columns:
        print("❌ Hata: TRY=X için fiyat verisi ('Close') çekilemedi.")
        usdtry_df = pd.DataFrame()
    else:
        # Timezone kaldırarak birleştir
        usdtry_df = usdtry_df_raw['Close'].tz_localize(None).to_frame(name='USDTRY_Kur')
        print("✅ TRY=X verisi çekildi.")

    # --- YENİ: BIST 100 EKLE ---
    print("\nFetching XU100.IS data...")
    xu100 = yf.Ticker("XU100.IS") # BIST 100 sembolü
    xu100_df_raw = xu100.history(period="3y")
    if xu100_df_raw.empty or 'Close' not in xu100_df_raw.columns:
        print("❌ Hata: XU100.IS için fiyat verisi ('Close') çekilemedi.")
        xu100_df = pd.DataFrame()
    else:
        # Timezone kaldırarak birleştir
        xu100_df = xu100_df_raw['Close'].tz_localize(None).to_frame(name='XU100_Endeks') # Yeni sütun adı
        print("✅ XU100.IS verisi çekildi.")
    # --- BIST 100 EKLEME BİTTİ ---

    # Proceed only if ALL DataFrames have data
    if not eregl_df.empty and not usdtry_df.empty and not xu100_df.empty:

        # --- GÜNCELLENDİ: Üç DataFrame'i birleştir ---
        # Önce EREGL ve USDTRY'yi birleştir
        df_temp = pd.merge(eregl_df, usdtry_df, left_index=True, right_index=True, how='inner')
        # Sonra sonucu XU100 ile birleştir
        df = pd.merge(df_temp, xu100_df, left_index=True, right_index=True, how='inner')
        # --- BİRLEŞTİRME GÜNCELLENDİ ---

        if df.empty:
             print("\n❌ Hata: Birleştirme sonrası DataFrame boş. Tarih aralıkları uyuşmuyor olabilir.")
        else:
            df.ffill(inplace=True)

            print("\nBirleştirilmiş Veri (Son 5 Satır):")
            print(df.tail()) # Artık 3 sütun olmalı

            # --- Görselleştirmeyi şimdilik yorum satırı yapalım (isteğe bağlı) ---
            # Üç değişkeni aynı grafikte göstermek karmaşık olabilir,
            # Regresyon sonucuna odaklanmak için bunu geçici olarak kapatabilirsin.
            """
            fig, ax1 = plt.subplots(figsize=(12, 6))
            # ... (Grafik kodu aynı kalabilir ama yorum dışı bırakabilirsin) ...
            plt.show()
            """
            print("\nGrafik çizimi (isteğe bağlı) yorum satırı yapıldı. Regresyon hücresine geçin.")
            # --- Görselleştirme Bitti ---
    else:
        print("\n❗️ Veri eksik olduğu için birleştirme ve analiz yapılamıyor.")

except Exception as e:
    print(f"Veri işleme hatası: {e}")

Fetching EREGL.IS data...
✅ EREGL.IS verisi çekildi.

Fetching TRY=X data...
✅ TRY=X verisi çekildi.

Fetching XU100.IS data...
✅ XU100.IS verisi çekildi.

Birleştirilmiş Veri (Son 5 Satır):
            EREGL_Fiyat  USDTRY_Kur  XU100_Endeks
Date                                             
2025-10-21    27.360001   41.928902  10467.200195
2025-10-22    26.840000   41.968201  10551.299805
2025-10-23    26.840000   41.976398  10608.299805
2025-10-24    27.240000   42.031101  10941.799805
2025-10-27    27.680000   41.992161  10853.400391

Grafik çizimi (isteğe bağlı) yorum satırı yapıldı. Regresyon hücresine geçin.


In [12]:
import statsmodels.api as sm # Ana kütüphane importu
import statsmodels.formula.api as smf # Formül API'si
import numpy as np # NaN kontrolü için

# 4. Fark Alınmış Değişkenlerle Çoklu Regresyon Modeli
try:
    # Önce 'df' DataFrame'inin var olup olmadığını kontrol et
    if 'df' not in locals() or df.empty:
        print("\nHATA: 'df' DataFrame bulunamadı veya boş. Lütfen önceki veri çekme hücresini başarıyla çalıştırın.")
    else:
        print("\nVerilerin farkları alınıyor (Günlük Değişimler Hesaplanıyor)...")
        # Tüm sütunların 1. farkını al
        df_diff = df.diff(1)

        # Fark alma işlemi ilk satırda NaN oluşturur, bu satırı atalım
        df_diff = df_diff.dropna()

        # Modeli kurmadan önce fark alınmış veride NaN kalıp kalmadığını kontrol et
        if df_diff.isnull().values.any():
             print("\nUYARI: Fark alınmış veride hala NaN değerler var. Regresyon çalıştırılamıyor.")
             print(df_diff[df_diff.isnull().any(axis=1)]) # NaN olan satırları göster
        elif df_diff.empty:
             print("\nUYARI: Fark alma sonrası DataFrame boş kaldı.")
        else:
            print("Fark alınmış verinin son 5 satırı:")
            print(df_diff.tail())

            print("\nFark Alınmış Değişkenlerle Regresyon Modeli Çalıştırılıyor...")
            # --- FORMÜL DEĞİŞKEN ADLARIYLA GÜNCELLENDİ ---
            # Sütun adları artık fark alınmış serilere karşılık gelmeli.
            # Not: Python'da değişken adlarında '~' gibi özel karakterler sorun yaratabilir.
            # Bu yüzden sütun adlarını değiştirelim.
            df_diff.rename(columns={
                'EREGL_Fiyat': 'EREGL_Fiyat_Diff',
                'USDTRY_Kur': 'USDTRY_Kur_Diff',
                'XU100_Endeks': 'XU100_Endeks_Diff'
            }, inplace=True)

            model_diff = smf.ols('EREGL_Fiyat_Diff ~ USDTRY_Kur_Diff + XU100_Endeks_Diff', data=df_diff).fit()
            # --- FORMÜL GÜNCELLENDİ ---

            print("\nFark Alınmış Model Özeti:")
            print(model_diff.summary())

            # Önemli değerleri ayıklama
            try:
                katsayi_kur_diff = model_diff.params['USDTRY_Kur_Diff']
                p_degeri_kur_diff = model_diff.pvalues['USDTRY_Kur_Diff']
            except KeyError: # Eğer model katsayıyı anlamsız bulup çıkarırsa
                 katsayi_kur_diff = np.nan
                 p_degeri_kur_diff = np.nan

            try:
                katsayi_xu100_diff = model_diff.params['XU100_Endeks_Diff']
                p_degeri_xu100_diff = model_diff.pvalues['XU100_Endeks_Diff']
            except KeyError:
                 katsayi_xu100_diff = np.nan
                 p_degeri_xu100_diff = np.nan

            adj_r_kare = model_diff.rsquared_adj
            dw_test = sm.stats.stattools.durbin_watson(model_diff.resid)

            print(f"\nUSDTRY Kur Değişim Katsayısı: {katsayi_kur_diff:.4f} (P-değeri: {p_degeri_kur_diff:.4f})")
            print(f"XU100 Endeks Değişim Katsayısı: {katsayi_xu100_diff:.4f} (P-değeri: {p_degeri_xu100_diff:.4f})")
            print(f"Adj. R-kare: {adj_r_kare:.4f}")
            print(f"Durbin-Watson Testi: {dw_test:.3f}")

            # Yorumlama
            print("\nDeğişimler Arasındaki İlişki Anlamlılığı:")
            if p_degeri_kur_diff < 0.05:
                print("- USDTRY Kurundaki günlük değişim ile EREGL fiyatındaki günlük değişim arasında istatistiksel olarak anlamlı bir ilişki var.")
            else:
                print("- USDTRY Kurundaki günlük değişim ile EREGL fiyatındaki günlük değişim arasında anlamlı bir ilişki bulunamadı.")

            if p_degeri_xu100_diff < 0.05:
                print("- XU100 Endeksindeki günlük değişim ile EREGL fiyatındaki günlük değişim arasında istatistiksel olarak anlamlı bir ilişki var.")
            else:
                print("- XU100 Endeksindeki günlük değişim ile EREGL fiyatındaki günlük değişim arasında anlamlı bir ilişki bulunamadı.")

            if dw_test < 1.5 or dw_test > 2.5:
                 print("\nUYARI: Durbin-Watson değeri (%.3f) hala ideal aralığın (1.5-2.5) dışında olabilir. Otokorelasyon kısmen devam ediyor olabilir." % dw_test)
            else:
                 print("\nDurbin-Watson değeri (%.3f) kabul edilebilir aralığa girmiş görünüyor (Otokorelasyon sorunu büyük ölçüde çözüldü)." % dw_test)

except NameError:
    print("\nHATA: 'df' DataFrame bulunamadı. Lütfen önceki veri çekme ve birleştirme hücresini başarıyla çalıştırdığınızdan emin olun.")
except Exception as e:
    print(f"\nRegresyon hatası: {e}")


Verilerin farkları alınıyor (Günlük Değişimler Hesaplanıyor)...
Fark alınmış verinin son 5 satırı:
            EREGL_Fiyat  USDTRY_Kur  XU100_Endeks
Date                                             
2025-10-21     0.180000   -0.001499    -17.200195
2025-10-22    -0.520000    0.039299     84.099609
2025-10-23     0.000000    0.008198     57.000000
2025-10-24     0.400000    0.054703    333.500000
2025-10-27     0.440001   -0.038940    -88.399414

Fark Alınmış Değişkenlerle Regresyon Modeli Çalıştırılıyor...

Fark Alınmış Model Özeti:
                            OLS Regression Results                            
Dep. Variable:       EREGL_Fiyat_Diff   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.471
Method:                 Least Squares   F-statistic:                     331.6
Date:                Tue, 28 Oct 2025   Prob (F-statistic):          1.27e-103
Time:                        14:32:31   Log-Likelihood:          